In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yolo-dataset/100_done/90019232302_2_v1_pg1.txt
/kaggle/input/yolo-dataset/100_done/90018818219_Proforma_pg1.txt
/kaggle/input/yolo-dataset/100_done/90019104930_Quotation_pg1.txt
/kaggle/input/yolo-dataset/100_done/90019012095_175379096_2_pg9.txt
/kaggle/input/yolo-dataset/100_done/90019028089_OTHERS_v1_pg1.txt
/kaggle/input/yolo-dataset/100_done/90019109032_OTHERS_v2_pg1.txt
/kaggle/input/yolo-dataset/100_done/90019272204_OTHERS_v1_pg1.txt
/kaggle/input/yolo-dataset/100_done/90018813452_OTHERS_v1_pg1.txt
/kaggle/input/yolo-dataset/100_done/90018777756_OTHERS_v1_pg1.txt
/kaggle/input/yolo-dataset/100_done/90018935306_1_v1_pg1.txt
/kaggle/input/yolo-dataset/100_done/90018912387_OTHERS_v2_pg1.txt
/kaggle/input/yolo-dataset/100_done/90019290546_1_v2_pg1.txt
/kaggle/input/yolo-dataset/100_done/90018862558_175238148_2_pg33.txt
/kaggle/input/yolo-dataset/100_done/90019109032_OTHERS_v1_pg1.txt
/kaggle/input/yolo-dataset/100_done/90018791802_1_v1_pg1.txt
/kaggle/input/yolo-dataset

In [4]:
from pathlib import Path
DATASET_ROOT = Path(r"/kaggle/input/yolo-dataset")  

In [8]:
from pathlib import Path

SPLIT_ROOT = Path("/kaggle/working/dataset_split")
SPLIT_ROOT.mkdir(parents=True, exist_ok=True)
data_yaml = SPLIT_ROOT / "data.yaml"

data_yaml.write_text(
f"""path: {SPLIT_ROOT.resolve().as_posix()}
train: images/train
val: images/val
test: images/test

names:
  0: signature
  1: stamp
""",
encoding="utf-8"
)

print("Wrote:", data_yaml)


Wrote: /kaggle/working/dataset_split/data.yaml


In [10]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
# ==== Notebook YOLO Fine-tune for Signature/Stamp (70/20/10) ====

import random
import shutil
from pathlib import Path
from typing import List, Tuple, Dict, Optional

# -------------------------
# USER SETTINGS (edit these)
# -------------------------
DATASET_ROOT = Path(r"/kaggle/input/yolo-dataset")     # <- change this to your dataset folder
SPLIT_ROOT   = Path(r"/kaggle/working/dataset_split")  # output split folder


SEED = 42
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.70, 0.20, 0.10

# YOLO training settings
PRETRAINED_MODEL = "yolov8n.pt"   # or yolov8s.pt
IMGSZ = 1024                      # docs benefit from 960-1280
EPOCHS = 80
BATCH = 8
DEVICE = "0"   # "0" for GPU0, or "cpu"
WORKERS = 4
PATIENCE = 25

# Test evaluation settings
CONF_THRESH = 0.25
IOU_THRESH = 0.50

# Classes (must match your label IDs)
CLASS_NAMES = {0: "signature", 1: "stamp"}

# -------------------------
# Helpers
# -------------------------
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def list_images(images_dir: Path) -> List[Path]:
    imgs = []
    for p in images_dir.rglob("*"):
        if p.is_file() and p.suffix.lower() in IMG_EXTS:
            imgs.append(p)
    return sorted(imgs)

def split_indices(n: int, seed: int, train_ratio: float, val_ratio: float, test_ratio: float):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-9
    idx = list(range(n))
    random.Random(seed).shuffle(idx)
    n_train = int(round(train_ratio * n))
    n_val   = int(round(val_ratio * n))
    n_train = min(n_train, n)
    n_val   = min(n_val, n - n_train)
    train_idx = idx[:n_train]
    val_idx   = idx[n_train:n_train+n_val]
    test_idx  = idx[n_train+n_val:]
    return train_idx, val_idx, test_idx

def copy_pair(img_path: Path, label_src: Optional[Path], img_dst: Path, label_dst: Path):
    ensure_dir(img_dst.parent)
    shutil.copy2(img_path, img_dst)

    ensure_dir(label_dst.parent)
    if label_src is not None and label_src.exists():
        shutil.copy2(label_src, label_dst)
    else:
        label_dst.write_text("", encoding="utf-8")  # empty label = negative sample

def read_yolo_labels(label_path: Path):
    if not label_path.exists():
        return []
    lines = [ln.strip() for ln in label_path.read_text(encoding="utf-8").splitlines() if ln.strip()]
    out = []
    for ln in lines:
        parts = ln.split()
        if len(parts) != 5:
            continue
        try:
            cls = int(parts[0])
            xc, yc, w, h = map(float, parts[1:])
            out.append((cls, xc, yc, w, h))
        except Exception:
            continue
    return out

def yolo_to_xyxy(norm_box, img_w: int, img_h: int):
    xc, yc, w, h = norm_box
    x1 = int((xc - w/2) * img_w)
    y1 = int((yc - h/2) * img_h)
    x2 = int((xc + w/2) * img_w)
    y2 = int((yc + h/2) * img_h)
    x1 = max(0, min(x1, img_w - 1))
    y1 = max(0, min(y1, img_h - 1))
    x2 = max(0, min(x2, img_w - 1))
    y2 = max(0, min(y2, img_h - 1))
    if x2 < x1: x1, x2 = x2, x1
    if y2 < y1: y1, y2 = y2, y1
    return (x1, y1, x2, y2)

def iou(a, b) -> float:
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2 - ix1), max(0, iy2 - iy1)
    inter = iw * ih
    area_a = max(0, ax2 - ax1) * max(0, ay2 - ay1)
    area_b = max(0, bx2 - bx1) * max(0, by2 - by1)
    denom = area_a + area_b - inter
    return (inter / denom) if denom > 0 else 0.0

def greedy_match(gt, pred, iou_thresh: float):
    """
    gt:   [(cls, box)]
    pred: [(cls, box, conf)]
    Return (TP, FP, FN) aggregated across classes (greedy matching per class).
    """
    tp = fp = fn = 0

    gt_by_cls: Dict[int, List[Tuple[int,int,int,int]]] = {}
    pr_by_cls: Dict[int, List[Tuple[Tuple[int,int,int,int], float]]] = {}

    for c, box in gt:
        gt_by_cls.setdefault(c, []).append(box)
    for c, box, conf in pred:
        pr_by_cls.setdefault(c, []).append((box, conf))

    classes = set(gt_by_cls.keys()) | set(pr_by_cls.keys())

    for c in classes:
        gt_boxes = gt_by_cls.get(c, [])
        pr_boxes = sorted(pr_by_cls.get(c, []), key=lambda x: x[1], reverse=True)
        matched = [False] * len(gt_boxes)

        for pb, _conf in pr_boxes:
            best_i, best_v = -1, 0.0
            for i, gb in enumerate(gt_boxes):
                if matched[i]:
                    continue
                v = iou(gb, pb)
                if v > best_v:
                    best_v, best_i = v, i
            if best_i >= 0 and best_v >= iou_thresh:
                tp += 1
                matched[best_i] = True
            else:
                fp += 1

        fn += matched.count(False)

    return tp, fp, fn

# -------------------------
# 1) Prepare split structure
# -------------------------
images_dir = DATASET_ROOT / "annotation"
labels_dir = DATASET_ROOT / "100_done"

assert images_dir.exists(), f"Missing: {images_dir}"
assert labels_dir.exists(), f"Missing: {labels_dir}"

images_all = list_images(images_dir)
print("Total images found:", len(images_all))
assert len(images_all) > 0, "No images found."

# keep ONLY images that have matching label file
images = []
label_paths = []
for img in images_all:
    lbl = labels_dir / f"{img.stem}.txt"
    if lbl.exists():
        images.append(img)
        label_paths.append(lbl)

print("Total labeled images used:", len(images))
assert len(images) > 0, "No labeled images found."

# split based on labeled images only
train_idx, val_idx, test_idx = split_indices(len(images), SEED, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

print(f"Split sizes: train={len(train_idx)} val={len(val_idx)} test={len(test_idx)}")

# clean + create split folders
shutil.rmtree(SPLIT_ROOT, ignore_errors=True)
ensure_dir(SPLIT_ROOT)

split_images_train = SPLIT_ROOT / "images" / "train"
split_images_val   = SPLIT_ROOT / "images" / "val"
split_images_test  = SPLIT_ROOT / "images" / "test"
split_labels_train = SPLIT_ROOT / "labels" / "train"
split_labels_val   = SPLIT_ROOT / "labels" / "val"
split_labels_test  = SPLIT_ROOT / "labels" / "test"

for p in [split_images_train, split_images_val, split_images_test,
          split_labels_train, split_labels_val, split_labels_test]:
    ensure_dir(p)

def do_copy(idxs, img_dst_dir, lbl_dst_dir):
    for i in idxs:
        img = images[i]
        lbl = label_paths[i]  # may be None
        copy_pair(
            img_path=img,
            label_src=lbl,
            img_dst=img_dst_dir / img.name,
            label_dst=lbl_dst_dir / f"{img.stem}.txt"
        )

do_copy(train_idx, split_images_train, split_labels_train)
do_copy(val_idx,   split_images_val,   split_labels_val)
do_copy(test_idx,  split_images_test,  split_labels_test)

# -------------------------
# 2) Write data.yaml
# -------------------------
data_yaml_path = SPLIT_ROOT / "data.yaml"
names_yaml = "\n".join([f"  {k}: {v}" for k, v in sorted(CLASS_NAMES.items())])

data_yaml_path.write_text(
f"""path: {SPLIT_ROOT.as_posix()}
train: images/train
val: images/val
test: images/test

names:
{names_yaml}
""",
encoding="utf-8"
)

print("Wrote:", data_yaml_path)

# -------------------------
# 3) Train YOLOv8
# -------------------------
from ultralytics import YOLO

model = YOLO(PRETRAINED_MODEL)

train_results = model.train(
    data=str(data_yaml_path),
    imgsz=IMGSZ,
    epochs=EPOCHS,
    batch=BATCH,
    device=DEVICE,
    workers=WORKERS,
    patience=PATIENCE,

    # mild, document-safe augments
    close_mosaic=10,
    mosaic=0.5,
    degrees=5.0,
    perspective=0.0005,
    scale=0.5,
    translate=0.05,
    fliplr=0.0,
    flipud=0.0,
)

# best path from trainer
best_path = None
try:
    best_path = Path(model.trainer.best)  # ultralytics stores best.pt path here
except Exception:
    pass

if not best_path or not best_path.exists():
    print("⚠️ Could not auto-locate best weights. Check runs/detect/*/weights/best.pt")
else:
    print("✅ Best weights:", best_path)

# -------------------------
# 4) Inference on test + Detection Error %
# -------------------------
from PIL import Image

best_model = YOLO(str(best_path)) if best_path else model

test_images = list_images(split_images_test)
print("Test images:", len(test_images))

total_tp = total_fp = total_fn = 0
per_cls = {c: {"tp": 0, "fp": 0, "fn": 0} for c in CLASS_NAMES.keys()}

for img_path in test_images:
    with Image.open(img_path) as im:
        w, h = im.size

    # GT boxes
    gt_label = split_labels_test / f"{img_path.stem}.txt"
    gt_norm = read_yolo_labels(gt_label)
    gt_boxes = [(cls, yolo_to_xyxy((xc, yc, bw, bh), w, h)) for (cls, xc, yc, bw, bh) in gt_norm]

    # Predictions
    preds = best_model.predict(source=str(img_path), imgsz=IMGSZ, conf=CONF_THRESH, verbose=False)
    pred_boxes = []
    for r in preds:
        if hasattr(r, "boxes") and r.boxes is not None:
            boxes = r.boxes
            for i in range(len(boxes)):
                b = boxes[i]
                raw_conf = getattr(b, "conf", 0.0)
                conf = float(raw_conf[0] if hasattr(raw_conf, "__len__") else raw_conf)
                raw_cls = getattr(b, "cls", 0)
                cls = int(raw_cls[0] if hasattr(raw_cls, "__len__") else raw_cls)
                xyxy = getattr(b, "xyxy", None)
                if xyxy is None:
                    continue
                coords = xyxy[0].tolist() if hasattr(xyxy[0], "tolist") else list(xyxy[0])
                x1, y1, x2, y2 = map(int, coords[:4])
                pred_boxes.append((cls, (x1, y1, x2, y2), conf))

    tp, fp, fn = greedy_match(gt_boxes, pred_boxes, IOU_THRESH)
    total_tp += tp
    total_fp += fp
    total_fn += fn

    # Per-class stats
    for c in CLASS_NAMES.keys():
        gt_c = [(cls, box) for cls, box in gt_boxes if cls == c]
        pr_c = [(cls, box, conf) for cls, box, conf in pred_boxes if cls == c]
        tp_c, fp_c, fn_c = greedy_match(gt_c, pr_c, IOU_THRESH)
        per_cls[c]["tp"] += tp_c
        per_cls[c]["fp"] += fp_c
        per_cls[c]["fn"] += fn_c

def prf(tp, fp, fn):
    prec = tp / (tp + fp) if (tp + fp) else 0.0
    rec  = tp / (tp + fn) if (tp + fn) else 0.0
    f1   = (2*prec*rec)/(prec+rec) if (prec+rec) else 0.0
    return prec, rec, f1

denom = total_tp + total_fp + total_fn
error_pct = 100.0 * (total_fp + total_fn) / denom if denom else 0.0
p, r, f1 = prf(total_tp, total_fp, total_fn)

print("\n=== TEST SUMMARY ===")
print(f"TP={total_tp}  FP={total_fp}  FN={total_fn}")
print(f"Precision={p:.3f}  Recall={r:.3f}  F1={f1:.3f}")
print(f"Detection Error % = {error_pct:.2f}%  (FP+FN)/(TP+FP+FN), IoU>={IOU_THRESH}, conf>={CONF_THRESH}")

print("\n=== PER-CLASS ===")
for c, name in CLASS_NAMES.items():
    tp_c, fp_c, fn_c = per_cls[c]["tp"], per_cls[c]["fp"], per_cls[c]["fn"]
    p_c, r_c, f1_c = prf(tp_c, fp_c, fn_c)
    denom_c = tp_c + fp_c + fn_c
    err_c = 100.0 * (fp_c + fn_c) / denom_c if denom_c else 0.0
    print(f"{name:9s}: TP={tp_c:4d} FP={fp_c:4d} FN={fn_c:4d} | P={p_c:.3f} R={r_c:.3f} F1={f1_c:.3f} | Err%={err_c:.2f}")


Total images found: 224
Total labeled images used: 98
Split sizes: train=69 val=20 test=9
Wrote: /kaggle/working/dataset_split/data.yaml
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset_split/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.5, multi_sc